# First Attempt Create Sphere array.

Try to create surfaces using meshes consisting of triangles. Using http://ipyvolume.readthedocs.io/en/latest/mesh.html as a source.

Load up both  `ipyvolume.pylab`and `numpy` 

In [66]:
import ipyvolume.pylab as p3
import numpy as np
import matplotlib.pyplot as plt

It looks like if we want to create surfaces and crystal structures, we'll want to be using surface plots. We can still call things such as spheres, here. The question is, can we control the shape/size/transparency of the spheres?

First, lets define a tetrahedral with $x$, $y$, and $z$ arrays defining the vertices. How does "triangles" know about the vertex indices? I guess this gets called during `p3.plot_trisurf`, where the arrays are called and then the drawing of the triangles is defined base on those arrays? E.g., "from 0 $\rightarrow$  1 $\rightarrow$  2" is "(1,0,-s) $\rightarrow$ (-1,0,-s) $\rightarrow$  (0,1,s)". That's hard-coded and could be permuted.

In [13]:
s = 1/2**0.5
# 4 vertices for the tetrahedron
x = np.array([1.,  -1, 0,  0])
y = np.array([0,   0, 1., -1])
z = np.array([-s, -s, s,  s])
# and 4 surfaces (triangles), where the number refer to the vertex index
triangles = [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1,3,2)]

In [14]:
p3.figure()
# we draw the tetrahedron
p3.plot_trisurf(x, y, z, triangles=triangles, color='orange')
# and also mark the vertices
p3.scatter(x, y, z, marker='sphere', color='blue')
p3.xyzlim(-1.5, 1.5)
p3.show()
print(x)

[ 1. -1.  0.  0.]


So, if I understand the `p3.scatter` command, we have sphere markers at each point in our array. We'll have to define our crystals in the correct $x$, $y$, $z$ order, then. That way of defining the array seems a bit backwards to me... check out what we can do with `p3.scatter` and all the other Widgety things we can do.

In [27]:
p3.scatter(x, y, z, marker='sphere', color='blue')


Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array…

Alright, great. I can make a tetrahedron. Let's try to make a simple cubic structure. Need 8 atoms  with $x,y,z$ positions:

| $x$ | $y$ | $z$ | $qrs$ |
| --- |---| ---| ---- |
| 0 |  0 | 0 | 000 |
| 1 |  0 | 0 | 100 |
| 0 |  1 | 0 | 010 |
| 0 |  0 | 1 | 001 |
| 1 |  1 | 0 | 110 |
| 1 |  0 | 1 | 101 |
| 0 |  1 | 1 | 011 |
| 1 |  1 | 1 | 111 |


In [16]:
x = np.array([0.,1.,0.,0.,1.,1.,0.,1.])
y = np.array([0.,0.,1.,0.,1.,0.,1.,1.])
z = np.array([0.,0.,0.,1.,0.,1.,1.,1.])
# Can I do edges using squares to make a unit cell? It doesn't seem so... the plot_trisurf function is tinangle-specific.
# squares = [(0, 1, 2, 3)]

p3.figure()
p3.scatter(x, y, z, marker='sphere', color='blue')
p3.xyzlim(0, 1)
p3.show()
print(x)

[0. 1. 0. 0. 1. 1. 0. 1.]


Alright, mild success. I've got a rotatable graphic with spheres located at the corners of a box. I need to figure out how to control:

- Size
- Labels
- Font
- Transparency (possible with scatter)
- Widget controls

I will quickly try to make a parameteric plot of a sphere because I think there may be limitations in what I can do with a scatter plot, but not with what I can do with a surface plot.

In [55]:
#Define mesh grid
r = 2
U = np.arange(-5, 5,0.25)
V = np.arange(-5, 5,0.25)
X, Y = np.meshgrid(U, V)
Z = np.sqrt(r**2 - X**2 -Y**2)

p3.figure()
p3.plot_surface(X, Z, Y, color="orange")
#p3.plot_wireframe(X, Z, Y, color="black")
p3.xyzlim(-5, 5)
p3.show()


C:\Users\jde621\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  


Ugh... messy because of the $\sqrt{}$ leading to imaginary numbers. Let's solve that with some smartness --- let's use spherical coordinates.

In [89]:
#Define mesh grid
r = 2
thetavec = np.arange(0,np.pi,np.pi/40)
phivec = np.arange(0,2*np.pi,np.pi/20)
th, ph = np.meshgrid(thetavec, phivec)

X = r*np.sin(th)*np.cos(ph)
Y = r*np.sin(th)*np.sin(ph)
Z = r*np.cos(th)

#Playing around with colormaps from matplotlib. But, I should figure out transparencies. Need to activate an alpha channel? 
#This doesn't seemt o be an option in the plot_surface function, but we were able to define color via cm in matplot lib,
#so in principle we can use the same idea to define an alpha channel... can't get it, though.
from matplotlib import cm
from matplotlib.colors import ListedColormap

colormap = cm.coolwarm
my_colormap = colormap(np.arange(colormap.N))
my_colormap[:,-1] = np.linspace(0,1,colormap.N)
my_colormap = ListedColormap(my_colormap)
znorm = Z - Z.min()
znorm /= znorm.ptp()
znorm.min(), znorm.max()
color = my_colormap(znorm)

p3.figure()
p3.plot_surface(X, Z, Y, color=color[...,0:3], wrapx=True, wrapy=False)
#Wrapping completes the curcuit, connecting beginning and endpoints.
#p3.plot_wireframe(X, Z, Y, color="black")
p3.xyzlim(-5, 5)
p3.show()